In [ ]:
IDENTIFIER = "beethoven-op67"
NUM = 4

import json
import networkx as nx
import pandas as pd
from dwave.system import DWaveSampler, FixedEmbeddingComposite

from plots import plotCSV
from qubo import createBQM
from samples import firstValid, totalEntropy, duplicates, overlaps


picklePath = f"../Pickles/{IDENTIFIER}/{IDENTIFIER}_"
numPath = f"../Pickles/{IDENTIFIER}/{NUM}/{IDENTIFIER}_{NUM}_"
dataPath = f"../Data/{IDENTIFIER}/{NUM}/{IDENTIFIER}_{NUM}_"

phrases = pd.read_csv(picklePath + "phrases.csv", index_col=[0,1])
G = nx.read_graphml(picklePath + "graph.graphml")
instruments = json.load(open(numPath + "instruments.json"))
embedding = json.load(open(numPath + "embedding.json", "r"))
qa = FixedEmbeddingComposite(DWaveSampler(), embedding)

In [ ]:
bqm = createBQM(G, phrases, instruments, 6, 12, 1)
sampleset = qa.sample(bqm, num_reads=100, label=f"{IDENTIFIER}_{NUM}")

In [ ]:
sampleset.to_pandas_dataframe(True).sort_values("energy").head()
s = sampleset.first.sample

print(duplicates(s, G))
print(overlaps(s, G))

#print(sampleset.filter(lambda d: duplicates(d.sample, G) == 0))
print(sampleset.info)

In [ ]:
def recordChainStrength(strengthRange, bqm, filepath: str) -> None:
    
    file = pd.DataFrame()
    reads = 500

    for s in strengthRange:
        sampleset = qa.sample(bqm, num_reads=reads, chain_strength=s, label=f"{IDENTIFIER}_{NUM}_cs={s}")
        sample = sampleset.first

        new_row = pd.DataFrame({
            #"Chain strength": [sampleset.info["embedding_context"]["chain_strength"]],
            "Chain strength": [s],
            "Total reads": [reads],
            "Time": [sampleset.info["timing"]["qpu_access_time"]],
            "Chain break fraction": [sample.chain_break_fraction],
            "Lowest energy": [bqm.energy(sample.sample)],
            "Total entropy": [totalEntropy(sample.sample, G)],
            "Overlaps": [overlaps(sample.sample, G)],
            "Duplicates": [duplicates(sample.sample, G)]
            })
    
        print(f"Chain strength {s} recorded!")
        file = pd.concat([file, new_row])

    file.to_csv(filepath, index=False, mode="a", header=False)

# 10 to 50 repeated 
chainStrengths = range(10,51)
bqm = createBQM(G, phrases, instruments, 6, 12, 1) # CHANGE

for _ in range(3):
    recordChainStrength(chainStrengths, bqm, dataPath+"chain-strength.csv")

`haydn-op1` = 25

`beethoven-op67` = 30